# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error

from tqdm import tqdm
import time
import joblib

In [2]:
X_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2017/WED/X_train.npy', allow_pickle=True)
y_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2017/WED/y_train.npy', allow_pickle=True)

X_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2017/WED/X_test.npy', allow_pickle=True)
y_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2017/WED/y_test.npy', allow_pickle=True)

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [29]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [3]:
np.unique(y_train, return_counts=True)

(array(['BENIGN', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest',
        'DoS slowloris', 'Heartbleed'], dtype=object),
 array([307762,   7265, 161014,   3876,   4059,      8]))

In [4]:
normal_data = X_train[y_train == 'BENIGN'] # normal class

# Helpers

In [5]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    recall = recall_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# OCSVM

In [6]:
from sklearn.svm import OneClassSVM

In [7]:
ocsvm = OneClassSVM(kernel='rbf', gamma='auto')

batch_size = 100
n_batches = int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

Training OCSVM: 100%|██████████| 3078/3078 [00:39<00:00, 77.44it/s] 


In [ ]:
ocsvm = OneClassSVM(kernel='linear', gamma='auto')

start_time = time.time()

ocsvm.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

In [8]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 'BENIGN', 'normal', 'anomaly')

In [9]:
evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)

{'model': ['ocsvm'],
 'accuracy': [0.7082295466598101],
 'precision': [0.6860870093888151],
 'recall': [0.997865164281293],
 'f1': [0.8131137416366443]}

In [10]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)) 
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.70823,0.686087,0.997865,0.813114


# SGDOCSVM

In [11]:
from sklearn.linear_model import SGDOneClassSVM

In [12]:
sgdocsvm = SGDOneClassSVM()

start_time = time.time()

sgdocsvm.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 0.559891939163208


In [13]:
y_pred_test_sgdocsvm = sgdocsvm.predict(X_test)
y_pred_test_sgdocsvm = np.where(y_pred_test_sgdocsvm == 1, 'normal', 'anomaly')
y_test_converted_sgdocsvm = np.where(y_test == 'BENIGN', 'normal', 'anomaly')

In [14]:
evaluate_model("sgdocsvm", y_test_converted_sgdocsvm, y_pred_test_sgdocsvm)

{'model': ['sgdocsvm'],
 'accuracy': [0.695188583733749],
 'precision': [0.9802665299846233],
 'recall': [0.5314983009739398],
 'f1': [0.6892738665683744]}

In [15]:
sgdocsvm_df = pd.DataFrame(evaluate_model("sgdocsvm", y_test_converted_sgdocsvm, y_pred_test_sgdocsvm)) 
sgdocsvm_df

,model,accuracy,precision,recall,f1
0,sgdocsvm,0.695189,0.980267,0.531498,0.689274


# LOF

In [16]:
from sklearn.neighbors import LocalOutlierFactor

In [17]:
start_time = time.time()

lof = LocalOutlierFactor(n_neighbors=50, contamination=0.001, novelty=True)
lof.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 26.538133144378662


In [18]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 'BENIGN', 'normal', 'anomaly')

In [19]:
evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)

{'model': ['lof'],
 'accuracy': [0.6364038118501615],
 'precision': [0.636482179078528],
 'recall': [0.9988757374025744],
 'f1': [0.7775259465385768]}

In [20]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)) 
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.636404,0.636482,0.998876,0.777526


# IF

In [21]:
from sklearn.ensemble import IsolationForest

In [22]:
start_time = time.time()

iforest = IsolationForest(n_estimators=200, max_samples=128, contamination=0.001, random_state=21)
iforest.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 59.34796667098999


In [23]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == "BENIGN", 'normal', 'anomaly')

In [24]:
evaluate_model("if", y_test_converted_if, y_pred_test_if)

{'model': ['if'],
 'accuracy': [0.635423529978948],
 'precision': [0.6358424725822532],
 'recall': [0.9989641625506865],
 'f1': [0.7770751959122509]}

In [25]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if)) 
if_df

,model,accuracy,precision,recall,f1
0,if,0.635424,0.635842,0.998964,0.777075


# PCA Reconstruction

In [26]:
from sklearn.decomposition import PCA

In [27]:
start_time = time.time()

pca = PCA(n_components=10)
pca.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 0.9824776649475098


In [28]:
normal_data_pca = pca.transform(normal_data)
normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

train_reconstruction_error = np.mean((normal_data - normal_data_reconstructed) ** 2, axis=1)

In [29]:
threshold = np.percentile(train_reconstruction_error, 95)

In [30]:
X_test_pca = pca.transform(X_test)
X_test_reconstructed = pca.inverse_transform(X_test_pca)

test_reconstruction_error = np.mean((X_test - X_test_reconstructed) ** 2, axis=1)

In [31]:
test_reconstruction_error

array([0.85440871, 2.14936658, 0.4279379 , ..., 0.12364499, 1.01218548,
       2.1430408 ])

In [32]:
y_pred_test_pca = (test_reconstruction_error > threshold).astype(int)
y_pred_test_pca = np.where(y_pred_test_pca == 1, 'normal', 'anomaly')
y_test_converted_pca = np.where(y_test == "BENIGN", 'normal', 'anomaly')

In [33]:
evaluate_model("pca", y_test_converted_pca, y_pred_test_pca)

{'model': ['pca'],
 'accuracy': [0.13715107589952916],
 'precision': [0.11267566974088714],
 'recall': [0.05185503328575218],
 'f1': [0.07102383321077901]}

In [34]:
pca_df = pd.DataFrame(evaluate_model("pca", y_test_converted_pca, y_pred_test_pca)) 
pca_df

,model,accuracy,precision,recall,f1
0,pca,0.137151,0.112676,0.051855,0.071024


# Results

In [35]:
results_df = pd.concat([ocsvm_df, sgdocsvm_df, lof_df, if_df, pca_df], ignore_index=True)
results_df.to_csv('/home/jovyan/ssl-ids/result/CIC-IDS-2017/results_cic_ids_2017_wed.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.708230,0.686087,0.997865,0.813114
1,sgdocsvm,0.695189,0.980267,0.531498,0.689274
2,lof,0.636404,0.636482,0.998876,0.777526
3,if,0.635424,0.635842,0.998964,0.777075
4,pca,0.137151,0.112676,0.051855,0.071024


In [36]:
joblib.dump(ocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/ocsvm.pkl')
joblib.dump(sgdocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/sgdocsvm.pkl')
joblib.dump(lof, '/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/lof.pkl')
joblib.dump(iforest, '/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/iforest.pkl')
joblib.dump(pca, '/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/pca.pkl')

['/home/jovyan/ssl-ids/model/CIC-IDS-2017/WED/pca.pkl']